In [1]:
import pandas as pd
import numpy as np
import re

# Input

In [2]:
D1 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor1_all_contig_annotations.csv'
D2 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor2_all_contig_annotations.csv'
D3 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor3_all_contig_annotations.csv'
D4 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor4_all_contig_annotations.csv'

In [3]:
O1 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor1_consensus_annotations.csv'
O2 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor2_consensus_annotations.csv'
O3 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor3_consensus_annotations.csv'
O4 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor4_consensus_annotations.csv'

In [4]:
C1 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor1_clonotypes.csv'
C2 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor2_clonotypes.csv'
C3 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor3_clonotypes.csv'
C4 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor4_clonotypes.csv'

# Output

In [5]:
ACA = '../../experiments/exp10x/run1/tcr/cellranger_tot/outs/multi/vdj_t/all_contig_annotations.csv'
CTS = '../../experiments/exp10x/run1/tcr/cellranger_tot/outs/per_sample_outs/cellranger_tot/vdj_t/clonotypes.csv'
COA = '../../experiments/exp10x/run1/tcr/cellranger_tot/outs/per_sample_outs/cellranger_tot/vdj_t/consensus_annotations.csv'

# Load

In [6]:
d1 = pd.read_csv(D1)
d2 = pd.read_csv(D2)
d3 = pd.read_csv(D3)
d4 = pd.read_csv(D4)

In [7]:
o1 = pd.read_csv(O1)
o2 = pd.read_csv(O2)
o3 = pd.read_csv(O3)
o4 = pd.read_csv(O4)

In [8]:
c1 = pd.read_csv(C1)
c2 = pd.read_csv(C2)
c3 = pd.read_csv(C3)
c4 = pd.read_csv(C4)

In [9]:
def get_cur_ct(df):
    return df.raw_clonotype_id.str.split('clonotype',expand=True)[1].fillna(0).astype(int)

def get_new_ct(df, mv_ct):
    cur_ct = get_cur_ct(df)
    new_ct = cur_ct.apply(lambda x: x+mv_ct if x!=0 else 0) #np.where(cur_ct != 0, cur_ct + mv_ct, 0)
    new_ct = 'clonotype' + new_ct.astype(str)
    return new_ct.replace('clonotype0',np.nan)

def get_dict(df, mv_ct):
    return pd.concat([df.raw_clonotype_id, get_new_ct(df, mv_ct)], axis=1).set_index('raw_clonotype_id')[1].to_dict()

In [10]:
#d1.raw_clonotype_id = np.where(d1.raw_clonotype_id != 'None', d1.raw_clonotype_id + '1', d1.raw_clonotype_id)
mv_ct = max(get_cur_ct(d1))
print(mv_ct)
dct = get_dict(d2, mv_ct)
c2.clonotype_id = c2.clonotype_id.map(dct)
o2.clonotype_id = o2.clonotype_id.map(dct)
d2.raw_clonotype_id = get_new_ct(d2, mv_ct)

mv_ct = max(get_cur_ct(d2))
print(mv_ct)
dct = get_dict(d3, mv_ct)
c3.clonotype_id = c3.clonotype_id.map(dct)
o3.clonotype_id = o3.clonotype_id.map(dct)
d3.raw_clonotype_id = get_new_ct(d3, mv_ct)

mv_ct = max(get_cur_ct(d3))
print(mv_ct)
dct = get_dict(d4, mv_ct)
c4.clonotype_id = c4.clonotype_id.map(dct)
o4.clonotype_id = o4.clonotype_id.map(dct)
d4.raw_clonotype_id = get_new_ct(d4, mv_ct)

30340
50993
69104


In [11]:
df = pd.concat([d1,d2,d3,d4], keys=['s1','s2','s3','s4'])

In [12]:
df.productive = df.productive == 'True'

In [13]:
df.raw_clonotype_id  #.str.split('clonotype').str[1]

s1  0         clonotype19
    1         clonotype19
    2         clonotype19
    3         clonotype19
    4         clonotype19
                 ...     
s4  309850            NaN
    309851            NaN
    309852            NaN
    309853            NaN
    309854            NaN
Name: raw_clonotype_id, Length: 1225332, dtype: object

In [14]:
df.loc[(df.raw_clonotype_id.isna()) & (df.cdr3 != 'None'), ['barcode','is_cell','high_confidence','chain','full_length','productive','cdr3']]

barcode  is_cell  high_confidence chain  full_length  \
s2 549     AAACCTGTCTCGAGTA-29     True             True   TRB         True   
   1352    AAAGATGAGGCTCAGA-16     True             True   TRB         True   
   6754    AACTCCCAGGCAATTA-40     True             True   TRA         True   
   9894    AAGCCGCGTTACCGAT-25     True             True   TRB         True   
   12812   AATCCAGTCGAGGTAG-15     True             True   TRB         True   
...                        ...      ...              ...   ...          ...   
s4 309850   TTTGTCATCTTGGGTA-5    False             True   TRB         True   
   309851   TTTGTCATCTTTACGT-1    False             True   TRB         True   
   309852   TTTGTCATCTTTACGT-1    False            False   TRB         True   
   309853   TTTGTCATCTTTAGGG-5    False             True   TRB         True   
   309854   TTTGTCATCTTTAGGG-5    False             True   TRA         True   

           productive                 cdr3  
s2 549          False          C*RQGWKHHIF  
   1352         False   CASSYNNRKLTMATPSVR  
   6754         False      CAVRL*PGRKAYLRT  
   9894         False     CQQGD*RGGQVDTQYF  
   12812        False       CASIRPLLIHPSTL  
...               ...                  ...  
s4 309850        True  CASSQALGLAGGPRNEQFF  
   309851        True      CASSARDRDTDTQYF  
   309852        True           CARFTREQFF  
   309853        True       CASSSGDRQNEQFF  
   309854        True     CATDGGIKAAGNKLTF  

[406577 rows x 7 columns]

In [15]:
df[df.barcode == 'AAACGGGTCTCAAACG-5']

barcode  is_cell                    contig_id  \
s3 480  AAACGGGTCTCAAACG-5     True  AAACGGGTCTCAAACG-5_contig_1   
   481  AAACGGGTCTCAAACG-5     True  AAACGGGTCTCAAACG-5_contig_2   
   482  AAACGGGTCTCAAACG-5     True  AAACGGGTCTCAAACG-5_contig_3   
   483  AAACGGGTCTCAAACG-5     True  AAACGGGTCTCAAACG-5_contig_4   
   484  AAACGGGTCTCAAACG-5     True  AAACGGGTCTCAAACG-5_contig_5   
s4 420  AAACGGGTCTCAAACG-5     True  AAACGGGTCTCAAACG-5_contig_1   
   421  AAACGGGTCTCAAACG-5     True  AAACGGGTCTCAAACG-5_contig_2   
   422  AAACGGGTCTCAAACG-5     True  AAACGGGTCTCAAACG-5_contig_3   

        high_confidence  length chain    v_gene d_gene   j_gene c_gene  \
s3 480             True     581   TRA    TRAV19   None   TRAJ29   TRAC   
   481             True     493   TRA    TRAV21   None   TRAJ39   TRAC   
   482             True     722   TRB    TRBV19   None  TRBJ1-1  TRBC1   
   483            False     617   TRB    TRBV19   None  TRBJ1-1  TRBC1   
   484            False     312   TRB      None   None  TRBJ2-3  TRBC2   
s4 420             True     532   TRA  TRAV13-1   None   TRAJ18   TRAC   
   421             True     486   TRB  TRBV10-1  TRBD1  TRBJ1-1  TRBC1   
   422            False     250   TRB      None   None  TRBJ2-3  TRBC2   

        full_length  productive              cdr3  \
s3 480         True        True   CALSEAFSGNTPLVF   
   481         True       False              None   
   482         True        True    CASSMSGPGTEAFF   
   483         True        True    CASSMSGPGTEAFF   
   484        False       False              None   
s4 420         True        True  CAPRGERGSTLGRLYF   
   421         True        True   CASRFSPGENTEAFF   
   422        False       False              None   

                                                 cdr3_nt  reads  umis  \
s3 480     TGTGCTCTGAGTGAGGCCTTCTCAGGAAACACACCTCTTGTCTTT   6588     5   
   481                                              None   3411     3   
   482        TGTGCCAGTAGTATGTCTGGACCCGGGACTGAAGCTTTCTTT   5712     6   
   483        TGTGCCAGTAGTATGTCTGGACCCGGGACTGAAGCTTTCTTT   1326     1   
   484                                              None    968     1   
s4 420  TGTGCTCCTAGAGGGGAGAGAGGCTCAACCCTGGGGAGGCTATACTTT   4728     7   
   421     TGCGCCAGCAGGTTTTCCCCAGGGGAGAACACTGAAGCTTTCTTT   2068     2   
   422                                              None    684     1   

       raw_clonotype_id         raw_consensus_id  
s3 480   clonotype51030  clonotype37_consensus_2  
   481   clonotype51030                     None  
   482   clonotype51030  clonotype37_consensus_1  
   483   clonotype51030                     None  
   484   clonotype51030                     None  
s4 420   clonotype69115  clonotype11_consensus_2  
   421   clonotype69115  clonotype11_consensus_1  
   422   clonotype69115                     None

In [16]:
df.reset_index(inplace=True)

In [17]:
df.barcode = df.barcode + '-' + df.level_0

In [18]:
df

,level_0,level_1,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,s1,0,AAACCTGAGACAAAGG-4-s1,True,AAACCTGAGACAAAGG-4_contig_1,True,722,TRB,TRBV10-3,TRBD2,TRBJ2-1,TRBC2,True,True,CAISDPGLAGGGGEQFF,TGTGCCATCAGTGACCCCGGACTAGCGGGAGGCGGGGGGGAGCAGT...,32237,18,clonotype19,clonotype19_consensus_1
1,s1,1,AAACCTGAGACAAAGG-4-s1,True,AAACCTGAGACAAAGG-4_contig_2,True,605,TRA,TRAV29DV5,None,TRAJ44,TRAC,True,True,CAASVSIWTGTASKLTF,TGTGCAGCAAGCGTTAGTATTTGGACCGGCACTGCCAGTAAACTCA...,6088,3,clonotype19,clonotype19_consensus_2
2,s1,2,AAACCTGAGACAAAGG-4-s1,True,AAACCTGAGACAAAGG-4_contig_3,True,738,TRA,TRAV8-6,None,TRAJ47,TRAC,True,True,CAAWDMEYGNKLVF,TGTGCCGCCTGGGACATGGAATATGGAAACAAGCTGGTCTTT,5358,3,clonotype19,clonotype19_consensus_3
3,s1,3,AAACCTGAGACAAAGG-4-s1,True,AAACCTGAGACAAAGG-4_contig_4,False,468,TRB,None,None,TRBJ2-3,TRBC2,False,False,None,None,2517,1,clonotype19,None
4,s1,4,AAACCTGAGACAAAGG-4-s1,True,AAACCTGAGACAAAGG-4_contig_5,False,488,TRB,None,None,TRBJ2-6,TRBC2,False,False,None,None,2468,1,clonotype19,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225327,s4,309850,TTTGTCATCTTGGGTA-5-s4,False,TTTGTCATCTTGGGTA-5_contig_1,True,579,TRB,TRBV4-3,TRBD2,TRBJ2-1,TRBC2,True,True,CASSQALGLAGGPRNEQFF,TGCGCCAGCAGCCAAGCCCTGGGACTAGCGGGGGGGCCAAGGAATG...,1444,2,NaN,None
1225328,s4,309851,TTTGTCATCTTTACGT-1-s4,False,TTTGTCATCTTTACGT-1_contig_1,True,537,TRB,TRBV6-1,TRBD1,TRBJ2-3,TRBC2,True,True,CASSARDRDTDTQYF,TGTGCCAGCAGTGCCCGGGACAGAGACACAGATACGCAGTATTTT,2145,1,NaN,None
1225329,s4,309852,TTTGTCATCTTTACGT-1-s4,False,TTTGTCATCTTTACGT-1_contig_2,False,492,TRB,TRBV10-3,None,TRBJ2-1,TRBC2,True,True,CARFTREQFF,TGTGCCAGGTTTACGCGTGAGCAGTTCTTC,1717,1,NaN,None
1225330,s4,309853,TTTGTCATCTTTAGGG-5-s4,False,TTTGTCATCTTTAGGG-5_contig_1,True,520,TRB,TRBV7-9,TRBD1,TRBJ2-1,TRBC2,True,True,CASSSGDRQNEQFF,TGTGCCAGCAGCTCAGGGGACAGACAAAATGAGCAGTTCTTC,889,1,NaN,None


In [19]:
df.to_csv(ACA, index=False)

In [107]:
cf = pd.concat([c1,c2,c3,c4])
cf.to_csv(CTS, index=False)

In [108]:
of = pd.concat([o1,o2,o3,o4])
of.to_csv(COA, index=False)

In [109]:
of

,clonotype_id,consensus_id,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis
0,clonotype10000,clonotype10000_consensus_1,723,TRB,TRBV2,None,TRBJ2-3,TRBC2,True,True,CASSEGDTQYF,TGTGCCAGCAGTGAAGGAGATACGCAGTATTTT,42044,10
1,clonotype10001,clonotype10001_consensus_1,698,TRB,TRBV11-1,TRBD1,TRBJ2-5,TRBC2,True,True,CASSLSPRQGLQETQYF,TGTGCCAGCAGCTTATCCCCCCGACAGGGCCTCCAAGAGACCCAGT...,49880,14
2,clonotype10001,clonotype10001_consensus_2,705,TRA,TRAV17,None,TRAJ49,TRAC,True,True,CATDPIGNQFYF,TGTGCTACGGACCCGATCGGTAACCAGTTCTATTTT,13649,4
3,clonotype10002,clonotype10002_consensus_1,449,TRA,TRAV2,None,TRAJ10,TRAC,True,True,CAVTGGGNKLTF,TGTGCTGTCACGGGAGGAGGAAACAAACTCACCTTT,1547,2
4,clonotype10002,clonotype10002_consensus_2,674,TRA,TRAV8-3,None,TRAJ5,TRAC,True,True,CALDGGRRALTF,TGTGCTCTAGACGGGGGCAGGAGAGCACTTACTTTT,3203,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33886,clonotype70103,clonotype999_consensus_1,850,TRA,TRAV8-2,None,TRAJ23,TRAC,True,True,CVVTSPRDNQGGKLIF,TGTGTTGTGACCTCACCGAGGGATAACCAGGGAGGAAAGCTTATCTTC,10254,11
33887,clonotype69203,clonotype99_consensus_1,840,TRA,TRAV8-3,None,TRAJ43,TRAC,True,True,CAVGAGNNDMRF,TGTGCTGTGGGTGCGGGGAACAATGACATGCGCTTT,35791,52
33888,clonotype69203,clonotype99_consensus_2,697,TRB,TRBV12-3,TRBD2,TRBJ2-5,TRBC2,True,True,CASSYPEGETQYF,TGTGCCAGCAGTTATCCGGAAGGCGAGACCCAGTACTTC,163646,170
33889,clonotype69113,clonotype9_consensus_1,756,TRA,TRAV34,None,TRAJ3,TRAC,True,True,CGADSRYSSASKIIF,TGTGGAGCAGACTCGCGGTACAGCAGTGCTTCCAAGATAATCTTT,893787,770
